# This code is for ModelB
build source model based on source countries data(CCPM only) and predict CCPM for target countries

In [4]:
import tensorflow as tf
# Importing the libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import os
from tqdm import tqdm
import datetime 

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.models import Model
from keras.models import load_model,clone_model

from keras.layers import Input, Embedding, LSTM, Dense,  Lambda

from keras.backend import slice
from keras.constraints import max_norm
import warnings
warnings.simplefilter('ignore')
os.environ["CUDA_VISIBLE_DEVICES"]='-1'

##　locate the directory storing the data 
os.chdir(os.getcwd()+'/data')

/data/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [6]:
def create_sequences_x(data, seq_length):
    xs = []
    for i in range(len(data)-seq_length+1):
        x = data[i:(i+seq_length)]
        xs.append(x)

    return np.array(xs)


def create_sequences_y(data, seq_length):
    ys = []
    for i in range(seq_length, len(data)):
        y = data[i]
        ys.append(y)
    return np.array(ys)

def MAPE(y, y_pred):
    mape = sum(abs(y-y_pred)/y)/len(y)
    print('MAPE: ', mape)
    return mape

# 1. Build the source model

## 1.1 Construct source  sequences

In [2]:
from os import listdir
from os.path import isfile, join
from re import sub
mypath = 'source/'
source_countries=['Austria','China (except Hubei)','Croatia','Germany','Hubei','Italy','Japan',
            'Lebanon','Monaco','Norway','Oman','United Arab Emirates']

In [5]:
pred_length = 7
seq_length = 7
x_seq0 = []
y_seq0 = []

for sc in source_countries:
    print(sc)
    # data preprocessing
    df = pd.read_excel('source/'+sc+'.xlsx', index_col=0)

    df_new_1day = df[['confirmed cases per million']].diff(periods=1)
    df_new_1day.rename(columns={'confirmed cases per million':'new cases'}, inplace=True)

    df_new_7days = df[['confirmed cases per million']].diff(periods=pred_length)
    df_new_7days.rename(columns={'confirmed cases per million':'new cases'}, inplace=True)

    df_new_7days['cum cases'] = 0
    df_new_7days['cum cases'][pred_length:] = df['confirmed cases per million'].values[0:len(df_new_7days)-pred_length]
    
    scaler_x = MinMaxScaler() #scale data into 0-1
    scaler_y = MinMaxScaler()

    if len(x_seq0)==0:
        x_seq0 = create_sequences_x(df_new_1day.dropna().to_numpy(), seq_length)
        y_seq0 = create_sequences_y(df_new_7days.dropna().to_numpy(), seq_length)
        x_seq0 = x_seq0[0:len(y_seq0)]
    else:
        tx_seq0 = create_sequences_x(df_new_1day.dropna().to_numpy(), seq_length)
        ty_seq0 = create_sequences_y(df_new_7days.dropna().to_numpy(), seq_length)
        tx_seq0 = tx_seq0[0:len(ty_seq0)]
        
        x_seq0 = np.concatenate((x_seq0, tx_seq0),axis=0)
        y_seq0 = np.concatenate((y_seq0, ty_seq0),axis=0)

12
Croatia
Norway
Lebanon
Hubei
Austria
China (except Hubei)
Oman
United Arab Emirates
Japan
Germany
Italy
Monaco


## 1.2 scale the sequence values

In [7]:
x_seq1 = np.reshape(x_seq0, newshape=(-1,1))

scaler_x = scaler_x.fit(x_seq1)
x = scaler_x.transform(x_seq1)
x = np.reshape(x, newshape=(x_seq0.shape))
y_seq1 = np.reshape(y_seq0[:,0:1], newshape=(-1,1))

scaler_y = scaler_y.fit(y_seq1)
y = scaler_y.transform(y_seq1)

y = np.reshape(y, newshape=(y_seq0[:,0:1].shape))

##  1.3 source model

In [10]:
seed(123)
train_idx = sample(range(len(x)),int(len(x)*0.8))
test_idx = set(range(len(x))).difference(train_idx)


X_train = x[train_idx].copy()
y_train = np.reshape(y[train_idx], (-1))

X_test = x[list(test_idx)].copy()
y_test = np.reshape(y[list(test_idx)], (-1))

# Building the RNN

main_input = Input(shape=(seq_length,1,), dtype='float32', name='main_input')  

lstm_out = LSTM(4)(main_input)  
main_output = Dense(units = 1)(lstm_out)
regressor = Model(inputs=main_input, outputs=main_output)

regressor.compile(optimizer='adam', loss='mse')


# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 9, batch_size = 16)#  10
regressor.save('../model/ModelB_nocontrol.pkl')


Epoch 1/8
714/714 [==============================] - 1s 2ms/step - loss: 0.0219
Epoch 2/8
714/714 [==============================] - 0s 553us/step - loss: 0.0186
Epoch 3/8
714/714 [==============================] - 0s 624us/step - loss: 0.0171
Epoch 4/8
714/714 [==============================] - 0s 583us/step - loss: 0.0157
Epoch 5/8
714/714 [==============================] - 0s 540us/step - loss: 0.0146
Epoch 6/8
714/714 [==============================] - 0s 573us/step - loss: 0.0137
Epoch 7/8
714/714 [==============================] - 0s 585us/step - loss: 0.0131
Epoch 8/8
714/714 [==============================] - 0s 501us/step - loss: 0.0127
MAPE:  1.6715718050490873
MAPE:  168.68416785978138


168.68416785978138

# 2. Target countries

In [9]:
from os import listdir
from os.path import isfile, join
from re import sub
mypath = 'target/'
target_countries = ['Albania','Algeria','Argentina','Armenia','Australia','Azerbaijan','Bangladesh','Belarus','Belgium','Bermuda',
             'Bolivia','Brazil','Bulgaria','Canada','Chile','Colombia','Costa Rica','Cuba','Czech Republic','Denmark',
             'El Salvador','Estonia','Finland','France','Ghana','Gibraltar','Greece','Honduras','Hungary','India',
             'Indonesia','Iran','Iraq','Ireland','Israel','Jamaica','Jordan','Liberia','Luxembourg','Malaysia','Mexico',
             'Morocco','Nepal','Netherlands','Nigeria','Pakistan','Paraguay','Peru','Philippines','Poland','Portugal',
             'Qatar','Republic of the Congo','Romania','Russia','Rwanda','Saudi Arabia','Senegal','Sierra Leone',
             'Singapore','Slovakia','Slovenia','South Africa','Sri Lanka','Switzerland','Thailand','Tunisia','Turkey',
             'Ukraine','United Kingdom','United States','Venezuela']

In [11]:
pred_model = load_model('../model/ModelB_nocontrol.pkl')
ret_test = pd.DataFrame(index=target_countries, columns=['MAPE'])
no_control = pd.DataFrame(index=['loop1','loop2','loop3','loop4','loop5'], columns=['MAPE(mean)','MAPE(std)','MAPE<0.1','MAPE<0.05'])

In [13]:
for l in range(5):
    for tar in target_countries:
        print(tar)
        # data preprocessing

        ## scale data
        df = pd.read_excel('target/'+tar+'.xlsx', index_col=0)

        df_new_1day = df.diff(periods=1)
        df_new_1day.rename(columns={'confirmed cases per million':'new cases'}, inplace=True)

        df_new_7days = df[['confirmed cases per million']].diff(periods=pred_length)
        df_new_7days.rename(columns={'confirmed cases per million':'new cases'}, inplace=True)

        df_new_7days['cum cases'] = 0
        df_new_7days['cum cases'][pred_length:] = df['confirmed cases per million'].values[0:len(df_new_7days)-pred_length]

        x_seq0 = create_sequences_x(df_new_1day.dropna().to_numpy(), seq_length)
        y_seq0 = create_sequences_y(df_new_7days.dropna().to_numpy(), seq_length)
        x_seq0 = x_seq0[0:len(y_seq0)]
        x_seq1 = np.reshape(x_seq0, newshape=(-1,1))

        x = scaler_x.transform(x_seq1)

        x = np.reshape(x, newshape=(x_seq0.shape))
        y_seq1 = np.reshape(y_seq0[:,0:1], newshape=(-1,1))
        y = scaler_y.transform(y_seq1)

        y = np.reshape(y, newshape=(y_seq0[:,0:1].shape))
        model =clone_model(pred_model)
        model.set_weights(pred_model.get_weights())
        test_idx = int(len(x)*0.8)

        X_train = x[0:test_idx, :, 0:1].copy()
        y_train = np.reshape(y[0:test_idx], (-1))

        X_test = x[test_idx:,:,0:1].copy()
        y_test = np.reshape(y[test_idx:], (-1))


        for layer in model.layers[:-1]:
            layer.trainable=False
        for layer in model.layers[-1:]:
            layer.trainable=True
        model.compile(optimizer='adam', loss='mse')
        # Fitting the RNN to the Training set
        model.fit(X_train, y_train, epochs = 7, batch_size = 4, verbose=0)# 原来10

        # Predicting daily cases
        predicted_cases = model.predict(X_test)
        predicted_cases = scaler_y.inverse_transform(predicted_cases)

        true_cases = np.reshape(scaler_y.inverse_transform(np.reshape(y_test,(-1,1))),(-1))+y_seq0[test_idx:,1]
        predicted_cases = np.reshape(predicted_cases,(-1))+y_seq0[test_idx:,1]


        mape = MAPE(true_cases, predicted_cases)
        ret_test.loc[tar, 'MAPE'] = mape
        
    no_control.loc['loop'+str(l+1), 'MAPE(mean)'] = ret_test.MAPE.mean() 
    no_control.loc['loop'+str(l+1), 'MAPE(std)'] = ret_test.MAPE.std()
    no_control.loc['loop'+str(l+1), 'MAPE<0.1'] = ret_test[(ret_test.MAPE<0.1)].shape[0]
    no_control.loc['loop'+str(l+1), 'MAPE<0.05']=ret_test[(ret_test.MAPE<0.05)].shape[0]

Iraq
MAPE:  0.1170193641803412
Denmark
MAPE:  0.04168017167168084
Republic of the Congo
MAPE:  0.31168769466453444
Slovakia
MAPE:  0.05124198961274342
Azerbaijan
MAPE:  0.08034460629248753
Australia
MAPE:  0.050189969084553004
Bolivia
MAPE:  0.19044346513275776
Bangladesh
MAPE:  0.09404682224713405
Indonesia
MAPE:  0.03467051292532779
Estonia
MAPE:  0.04035225086349593
Rwanda
MAPE:  0.1097237846244324
Nigeria
MAPE:  0.2004347865380084
Bermuda
MAPE:  0.03000449197744275
Jordan
MAPE:  0.039047787775498714
Argentina
MAPE:  0.16509828663195478
Finland
MAPE:  0.04418868529768805
Paraguay
MAPE:  0.0704873283773625
Saudi Arabia
MAPE:  0.06720963706623531
Turkey
MAPE:  0.04355456795483205
Ireland
MAPE:  0.03349578921839156
Iran
MAPE:  0.017478653125681927
Switzerland
MAPE:  0.042427358132972326
Colombia
MAPE:  0.11363012543007756
Albania
MAPE:  0.02317180880707509
Pakistan
MAPE:  0.08181637016683428
Belgium
MAPE:  0.026522221387351403
United States
MAPE:  0.008611427075104686
Israel
MAPE:  0.0

In [15]:
no_control.to_csv('../result/ModelB_nocontrol.csv')